In [30]:
import os
import sys
sys.path.append('..')

from scripts.data_processing import load_processed_recipes, load_raw_recipes

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

## Load the data

In [17]:
# Load the data
recipes_df = load_processed_recipes(data_folder_path=os.path.join('..', 'data', 'processed'))
recipes_df

,id,minutes,n_steps,calories,fat_amount,sodium_amount,protein_amount,sugar_amount,saturated_fat,carbohydrates,...,tomatoes,unsalted butter,vanilla,vanilla extract,vegetable oil,vinegar,walnuts,water,worcestershire sauce,zucchini
0,38,1485,13,170.9,3.0,120.0,1.0,6.0,6.0,12.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,39,265,17,1110.7,90.0,81.0,15.0,126.0,82.0,28.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,40,35,8,311.1,0.0,308.0,0.0,0.0,0.0,27.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,41,1460,18,536.1,36.0,128.0,64.0,58.0,19.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,43,140,35,437.9,29.0,170.0,11.0,13.0,54.0,19.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186093,389129,25,11,427.5,36.0,9.0,5.0,93.0,16.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186094,389130,37,11,203.5,12.0,94.0,8.0,5.0,5.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
186095,389131,23,14,91.3,10.0,28.0,2.0,1.0,18.0,2.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
186096,389133,70,6,747.6,83.0,13.0,8.0,25.0,160.0,17.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Normalize the data

In [18]:
scale = StandardScaler()
recipes_df_norm = recipes_df.copy()
columns_to_scale = ['minutes', 'n_steps', 'calories', 'fat_amount', 'sodium_amount', 'protein_amount', 'sugar_amount', 'saturated_fat', 'carbohydrates']
recipes_df_norm[columns_to_scale] = scale.fit_transform(recipes_df_norm[columns_to_scale])
recipes_df_norm

,id,minutes,n_steps,calories,fat_amount,sodium_amount,protein_amount,sugar_amount,saturated_fat,carbohydrates,...,tomatoes,unsalted butter,vanilla,vanilla extract,vegetable oil,vinegar,walnuts,water,worcestershire sauce,zucchini
0,38,-0.002044,0.542541,-0.246838,-0.425846,0.037382,-0.246240,-0.499553,-0.409922,-0.044674,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,39,-0.002289,1.226733,0.484522,0.651483,-0.006700,-0.130776,1.520152,0.343333,0.132704,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,40,-0.002335,-0.312699,-0.137733,-0.462995,0.249877,-0.254488,-0.600538,-0.469389,0.121618,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,41,-0.002049,1.397781,0.037364,-0.017204,0.046424,0.273348,0.375653,-0.281076,0.055101,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,43,-0.002314,4.305597,-0.039057,-0.103885,0.093896,-0.163766,-0.381736,0.065818,0.032929,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186093,389129,-0.002337,0.200445,-0.047150,-0.017204,-0.088081,-0.213251,0.964733,-0.310810,-0.166622,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186094,389130,-0.002335,0.200445,-0.221469,-0.314398,0.007994,-0.188508,-0.516383,-0.419833,-0.066847,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
186095,389131,-0.002338,0.713589,-0.308784,-0.339164,-0.066605,-0.237993,-0.583707,-0.290987,-0.155536,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
186096,389133,-0.002328,-0.654795,0.201955,0.564801,-0.083559,-0.188508,-0.179766,1.116410,0.010756,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Calculate Similarity between recipes

In [26]:
# Use of a subset right now
subest_recipes_df = recipes_df_norm.iloc[:100, :]
# Build similarity marrix of movies based on similarity of genres
csmatrix = cosine_similarity(subest_recipes_df)
csmatrix = pd.DataFrame(csmatrix,columns=subest_recipes_df['id'],index=subest_recipes_df['id'])
csmatrix.head()

id,38,39,40,41,43,45,48,49,50,52,...,207,209,212,213,215,216,223,224,226,229
id,,,,,,,,,,,,,,,,,,,,,
38,1.000000,0.993135,0.998412,0.997251,0.993387,0.997475,0.994331,0.996211,0.997216,0.998368,...,0.998957,0.998879,0.998927,0.998987,0.998468,0.998384,0.998885,0.998957,0.999091,0.998938
39,0.993135,1.000000,0.993135,0.993196,0.991388,0.992666,0.992831,0.994599,0.992596,0.992582,...,0.994379,0.994662,0.994501,0.994519,0.995097,0.994857,0.994502,0.994620,0.994774,0.994639
40,0.998412,0.993135,1.000000,0.995956,0.991886,0.998118,0.993245,0.995298,0.998012,0.999356,...,0.999062,0.999123,0.999155,0.999120,0.998728,0.998432,0.999011,0.999174,0.998943,0.999138
41,0.997251,0.993196,0.995956,1.000000,0.993954,0.995904,0.994646,0.995992,0.994849,0.996292,...,0.997159,0.997233,0.997140,0.997239,0.997195,0.996982,0.997211,0.997166,0.997311,0.997281
43,0.993387,0.991388,0.991886,0.993954,1.000000,0.992715,0.997897,0.994834,0.991393,0.991233,...,0.992753,0.993084,0.993064,0.993313,0.992814,0.992872,0.992970,0.993063,0.993611,0.993417


In [53]:
def most_similar_recipe(recipes_df, recipes_df_processed, recipe_id, top_n=10):
    top_recipe_names = []
    top_recipe_ids = []
    
    # Get the name of the recipe from the id
    recipe_name = recipes_df[recipes_df['id'] == recipe_id]['name']
    
    # Normalize the processed data
    scale = StandardScaler()
    columns_to_scale = ['minutes', 'n_steps', 'calories', 'fat_amount', 'sodium_amount', 'protein_amount', 'sugar_amount', 'saturated_fat', 'carbohydrates']
    recipes_df_norm = recipes_df_processed.copy()
    recipes_df_norm[columns_to_scale] = scale.fit_transform(recipes_df_processed[columns_to_scale])
    
    # Build the similarity matrix of recipes
    csmatrix = cosine_similarity(recipes_df_norm)
    csmatrix = pd.DataFrame(csmatrix,columns=recipes_df_norm['id'],index=recipes_df_norm['id'])
    
    # Get the top n most similar recipes
    most_similar_recipes = csmatrix.loc[recipe_id, :].sort_values(ascending=False).index[1:top_n+1]
    for recipe in most_similar_recipes:
        top_recipe_names.append(recipes_df[recipes_df['id'] == recipe]['name'])
        top_recipe_ids.append(recipe)
    
    # Return the top names and ids
    return top_recipe_names, top_recipe_ids   
    

In [56]:
recipes_df = load_raw_recipes(data_folder_path=os.path.join('..', 'data'))
recipes_df_processed = load_processed_recipes(data_folder_path=os.path.join('..', 'data', 'processed'))
subset_df = recipes_df_processed.iloc[:100, :]

top_recipes, top_recipe_ids = most_similar_recipe(recipes_df, subset_df, 39, top_n=10)
print(f"Top Recipe name: {recipes_df[recipes_df['id'] == 39]['name']}")
display(top_recipes)
display(top_recipe_ids)

Top Recipe name: 23695    biryani
Name: name, dtype: object


[46956    chicken fried steak with cracked pepper gravy
 Name: name, dtype: object,
 32064    butterflied lamb with garlic butter
 Name: name, dtype: object,
 54270    citrus baked halibut
 Name: name, dtype: object,
 54546    clam   lobster bake
 Name: name, dtype: object,
 2012    a new spaghetti with clams
 Name: name, dtype: object,
 53707    cinnamon curry rice
 Name: name, dtype: object,
 42595    chicken and broccoli lasagna
 Name: name, dtype: object,
 36729    carrie s pizza rolls
 Name: name, dtype: object,
 49636    chocolate cashew spread
 Name: name, dtype: object,
 65682    crock pot chocolate zucchini nut bread
 Name: name, dtype: object]

[187, 109, 201, 196, 115, 205, 112, 141, 216, 215]

## Generate Predicted Ratings

In [57]:
recipes_df[recipes_df['id'] == 39]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
23695,biryani,39,265,1567,1999-08-29,"['weeknight', 'time-to-make', 'course', 'main-...","[1110.7, 90.0, 81.0, 15.0, 126.0, 82.0, 28.0]",17,['soak saffron in warm milk for 5 minutes and ...,"delhi, india","['saffron', 'milk', 'hot green chili peppers',...",26
